In [ ]:
# Kaggle competition

# Background

# Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression 

from sklearn.model_selection import train_test_split, cross_validate, \
cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import confusion_matrix, silhouette_score, roc_curve, auc, \
RocCurveDisplay, classification_report, ConfusionMatrixDisplay 

from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, \
StackingClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

from pathlib import Path
import joblib

import xgboost
import time

import warnings
warnings.filterwarnings('ignore')

# Data Understanding

## Downloading data files and unzipping locally

In [ ]:
# https://www.kaggle.com/competitions/tlvmc-parkinsons-freezing-gait-prediction/overview

# ! unzip tlvmc-parkinsons-freezing-gait-prediction.zip

## Reading csv's

In [4]:
tasks = pd.read_csv('tlvmc-parkinsons-freezing-gait-prediction/tasks.csv')
tasks

,Id,Begin,End,Task
0,02ab235146,10.000,190.480,Rest1
1,02ab235146,211.240,271.560,Rest2
2,02ab235146,505.880,522.400,4MW
3,02ab235146,577.960,594.640,4MW-C
4,02ab235146,701.320,715.280,MB1
...,...,...,...,...
2812,f9fc61ce85,800.586,810.714,TUG-DT
2813,f9fc61ce85,904.416,926.181,Turning-ST
2814,f9fc61ce85,959.211,989.855,Turning-DT
2815,f9fc61ce85,1087.313,1105.086,Hotspot1


In [5]:
subjects = pd.read_csv('tlvmc-parkinsons-freezing-gait-prediction/subjects.csv')
subjects

,Subject,Visit,Age,Sex,YearsSinceDx,UPDRSIII_On,UPDRSIII_Off,NFOGQ
0,04fcdb,1.0,63,M,3.0,30.0,NaN,0
1,05595e,1.0,56,M,8.0,28.0,NaN,0
2,0967b2,1.0,59,M,10.0,38.0,48.0,19
3,0967b2,2.0,59,M,10.0,37.0,44.0,13
4,097078,NaN,70,F,10.0,27.0,50.0,20
...,...,...,...,...,...,...,...,...
168,f90887,1.0,72,M,16.0,35.0,46.0,26
169,fc1e1b,1.0,82,F,11.0,38.0,42.0,21
170,fe5d84,2.0,72,M,14.0,32.0,45.0,17
171,fe5d84,1.0,72,F,14.0,13.0,33.0,15


In [7]:
events = pd.read_csv('tlvmc-parkinsons-freezing-gait-prediction/events.csv')
events

,Id,Init,Completion,Type,Kinetic
0,003f117e14,8.61312,14.7731,Turn,1.0
1,009ee11563,11.38470,41.1847,Turn,1.0
2,009ee11563,54.66470,58.7847,Turn,1.0
3,011322847a,28.09660,30.2966,Turn,1.0
4,01d0fe7266,30.31840,31.8784,Turn,1.0
...,...,...,...,...,...
3707,f9fc61ce85,628.56000,631.6650,Walking,0.0
3708,f9fc61ce85,782.49800,782.6530,Walking,1.0
3709,f9fc61ce85,931.93900,933.4470,Turn,1.0
3710,f9fc61ce85,990.85900,991.8580,Turn,0.0


## Exploring data

### tDCS data

In [ ]:
tlvmc-parkinsons-freezing-gait-prediction

In [ ]:
# can replace this pathname with the full path to the folder locally
path_feat = r'C:\Users\JonMc\Documents\Flatiron\' 

# Get the files from the path provided
files_feat = Path(path_feat).glob('*.csv')

In [ ]:
# this for loop will create a separate column ('File') based on the filename, to separate subjects if needed

dfs_1 = []
for f in files_feat:
    data = pd.read_csv(f)
    # .stem is method for pathlib objects to get the filename without the extension
    data['File'] = f.stem
    dfs_1.append(data)

In [ ]:
# concatenating all 17 files into one dataframe
df_feat = pd.concat(dfs_1, ignore_index=True)